#### Single model data analysis

In [ ]:
from data import data

import pickle

import matplotlib.lines as mlines

import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
area_names = data.mrdh65_to_name

folder = ""
image_folder = "results"
image_name = "base1"
load_names = ["base1"]
runs = ["base1"]

journeys_dict = {}
uxsim_dict = {}
parked_dict = {}

for run, load_name in zip(runs, load_names):
    with open(f"../results/{folder}uxsim_df_{load_name}.pkl", "rb") as f:
        uxsim_dict[run] = pickle.load(f)
    with open(f"../results/{folder}parked_dict_{load_name}.pkl", "rb") as f:
        parked_dict[run] = pickle.load(f)
    journeys_dict[run] = pd.read_feather(f"../results/{folder}journeys_df_{load_name}.feather")

In [ ]:
city_areas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 28, 29, 31, 34, 41, 43, 44, 45]
city_area_names = [area_names[area] for area in city_areas]

trips_by_hour_chances = pd.read_pickle("../data/trips_by_hour_chances.pickle")

start_time, end_time = int(journeys_dict[runs[0]]['start_time'].min()), int(journeys_dict[runs[0]]['start_time'].max())
end_time = 22
print(f"Start time: {start_time}, End time: {end_time}")

In [ ]:
# Throw away the first 15 minutes. Road network is not fully loaded yet, so travel times are often 0.
for run, journeys_df in journeys_dict.items():
    journeys_dict[run] = journeys_df[journeys_df['start_time'] > start_time + 0.25]

### Journeys data

In [ ]:
# Print the mode choice distribution
mode_counts = {}
mode_counts_weighted = {}
journey_counts = {}
journey_counts_norm = {}

for run, journeys_df in journeys_dict.items():
    mode_counts[run] = journeys_df['mode'].value_counts(normalize=True).to_dict()
    print(f"{run} Mode choice distribution: {({mode: f"{count:.2%}" for mode, count in mode_counts[run].items()})}")
    
    # Distance weighted mode choice distribution
    mode_counts_weighted[run] = journeys_df.groupby('mode', observed=True)['distance'].sum() / journeys_df['distance'].sum()
    print(f"{run} Distance weighted mode choice distribution: {({mode: f"{count:.2%}" for mode, count in mode_counts_weighted[run].items()})}")
    
    # start_time hour (float to int)
    journeys_df['start_time_h'] = journeys_df['start_time'].astype(int)
    
    # Group by start_time
    journeys_grouped = journeys_df.groupby(['start_time_h'])
    journey_counts[run] = journeys_grouped['mode'].value_counts(normalize=False).unstack().fillna(0)
    # Get the percentage for each mode of each hour in a dataframe
    journey_counts_norm[run] = journeys_grouped['mode'].value_counts(normalize=True).unstack().fillna(0)
    
# Convert journey_counts and journey_counts_norm into multi-indexed DataFrames
journey_counts_df = pd.concat(journey_counts, axis=0, keys=journey_counts.keys(), names=["Run", "Hour"]).stack().reset_index()
journey_counts_norm_df = pd.concat(journey_counts_norm, axis=0, keys=journey_counts_norm.keys(), names=["Run", "Hour"]).stack().reset_index()

# Plot with seaborn
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs = axs.flatten()

run_line_styles = ["solid", "dashed", "dashdot", "dotted"]
run_line_styles = {run: style for run, style in zip(runs, run_line_styles)}

for i, df in enumerate([journey_counts_df, journey_counts_norm_df]):
    for run in runs:
        dfr = df[df['Run'] == run]
        style = run_line_styles[run]
        sns.lineplot(data=dfr, x='Hour', y=0, hue='mode', linestyle=style, ax=axs[i])
    axs[i].set_title(f"Mode choice distribution over time ({'Absolute' if i == 0 else 'Normalized'})")
    axs[i].set_ylabel('Percentage')
    axs[i].set_xlabel('Time of day (hour)')
    axs[i].set_xlim(start_time, end_time)
    axs[i].set_ylim(bottom=0)
    if i == 1:
        axs[i].get_legend().remove()

# Custom legends
run_legend_lines = [mlines.Line2D([], [], color='black', linestyle=run_line_styles[run], label=f'Run {run}') for run in runs]
mode_colors = sns.color_palette()
mode_legend_lines = [mlines.Line2D([], [], color=color, linestyle='-', label=mode) for color, mode in zip(mode_colors, df['mode'].unique())]
custom_legend = run_legend_lines + mode_legend_lines
axs[0].legend(handles=custom_legend, title='Legend', loc='upper right')

plt.tight_layout()
plt.savefig(f'../img/{image_folder}/mode_distribution_{image_name}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
for run, journeys_df in journeys_dict.items():
    # Local analysis. Select the 65x65 areas ['Noord', 'Kralingen', 'Rotterdam Centrum', 'Feyenoord', 'Delfshaven'] from journeys_df.mrdh65. Use the data.mrdh65_to_name dictionary to map the area numbers to names.
    inner_rotterdam = ['Noord', 'Kralingen', 'Rotterdam Centrum', 'Feyenoord', 'Delfshaven']
    # origin is in pc4, use data.pc4_to_mrdh65_city to map to mrdh65
    journeys_df["origin_mrdh65_name"] = journeys_df["origin"].map(data.pc4_to_mrdh65_city).map(data.mrdh65_to_name)
    journeys_df["destination_mrdh65_name"] = journeys_df["destination"].map(data.pc4_to_mrdh65_city).map(data.mrdh65_to_name)
    
    # Filter the area index over city_areas
    journeys_df_rc = journeys_df[(journeys_df['origin_mrdh65_name'].isin(inner_rotterdam)) & (journeys_df['destination_mrdh65_name'].isin(inner_rotterdam))]
    print(f"Selected {len(journeys_df_rc)} journeys from the inner Rotterdam areas ({len(journeys_df_rc) / len(journeys_df):.2%}) of the total journeys.")
    
    # Print the mode choice distribution
    mode_counts = journeys_df_rc['mode'].value_counts(normalize=True).to_dict()
    print(f"Mode choice distribution: {({mode: f"{count:.2%}" for mode, count in mode_counts.items()})}")
    
    # Distance weighted mode choice distribution
    mode_counts_weighted = journeys_df_rc.groupby('mode', observed=True)['distance'].sum() / journeys_df_rc['distance'].sum()
    print(f"Distance weighted mode choice distribution: {({mode: f"{count:.2%}" for mode, count in mode_counts_weighted.items()})}\n")

In [ ]:
height = len(runs) * 4
fig, axs = plt.subplots(len(runs), 4, figsize=(16, height))
if len(runs) == 1:
    axs = axs[np.newaxis, :]

color_dict = {'bike': 'tab:blue', 'car': 'tab:orange', 'transit': 'tab:green', 'av': 'tab:red'}

for j, (run, journeys_df) in enumerate(journeys_dict.items()):
    journeys_df['travel_time_min'] = journeys_df['travel_time'] / 60
    # Sort by mode, in this order: bike, car, transit, av
    journeys_df = journeys_df.sort_values('mode', key=lambda x: x.map({'bike': 0, 'car': 1, 'transit': 2, 'av': 3}))
    
    n_bins = 40
    hist_columns = ['travel_time_min', 'distance', 'cost', 'perceived_cost']
    x_labels = ['Travel time (min)', 'Distance (km)', 'Cost (€)', 'Perceived cost (€)']
    upper_xlim = [journeys_df[col].quantile(0.99) for col in hist_columns]
    
    # Histogram of each column
    
    for i, col in enumerate(hist_columns):
        plot_df = journeys_df[(journeys_df[col] < upper_xlim[i]) & (journeys_df[col] > 0)][[col, 'mode']].copy()
        sns.histplot(plot_df, x=col, hue='mode', multiple='stack', ax=axs[j, i], bins=n_bins, palette=color_dict)
        if j == 0:
            axs[j, i].set_title(f'{col} distribution')
        axs[j, i].set_xlabel(x_labels[i])
        axs[j, i].set_ylabel('Frequency')
        axs[j, i].set_xlim(0, upper_xlim[i])

plt.tight_layout()
plt.savefig(f'../img/{image_folder}/journeys_data_{image_name}.png', dpi=300, bbox_inches='tight')
plt.show()

### Input data visualization (for comparison)

In [ ]:
# In the base run, check if there are any travel times by car under 40 seconds
journeys_base = journeys_dict["base"]
# Drop the first 15 minutes
journeys_base = journeys_base[journeys_base['start_time'] > 5.15]
# journeys_base_car = journeys_base[journeys_base['mode'] == 'car']
journeys_base_car = journeys_base[journeys_base['travel_time'] < 60]
journeys_base_car

In [ ]:
# For a weekday, take the average of days 0-3 (Monday-Thursday)
trips_by_hour_chance = trips_by_hour_chances.iloc[:, 0:4].mean(axis=1).drop("Total")
# Drop the hours that are not in the range of the model and save as a dictionary
trips_by_hour_chance = trips_by_hour_chance.loc[start_time:(end_time)]
# Set column name
trips_by_hour_chance.name = 'Chance'
# To df
trips_by_hour_chance = trips_by_hour_chance.reset_index()
# Set hour as int
trips_by_hour_chance['Hour'] = trips_by_hour_chance['Hour'].astype(int)
trips_by_hour_chance.head(3)

In [ ]:
# Plot trips_by_hour_chances series
fig, ax = plt.subplots(figsize=(6, 3))
sns.barplot(data=trips_by_hour_chance, x='Hour', y='Chance', ax=ax)
ax.set_title('Trip chances per Agent per hour')
ax.set_ylabel('Chance of taking trip')
ax.set_xlabel('Time of day (hour)')
# Save
plt.savefig(f'../img/input_data.png', dpi=300, bbox_inches='tight')

### UXsim data analysis

In [ ]:
for run, uxsim_df in uxsim_dict.items():
    # time_bin (first level multi index) from seconds to hours
    uxsim_df.index = pd.MultiIndex.from_tuples([(time/3600+start_time, area) for time, area in uxsim_df.index], names=['time_bin', 'area'])
    
    # Filter the area index over city_areas
    uxsim_df = uxsim_df.loc[(slice(None), city_areas), :]

In [ ]:
# # Plot
# fig, axs = plt.subplots(3, 2, figsize=(12, 12))
# axs = axs.flatten()
# 
# plot_run = "new_08"
# for i, variable in enumerate(uxsim_dict[plot_run].columns):
#     sns.lineplot(data=uxsim_df, x='time_bin', y=variable,  ax=axs[i], errorbar=("pi", 50), hue='area', palette='rocket')
#     axs[i].set_title(f"{variable} in different areas")
#     axs[i].set_ylabel(variable)
#     axs[i].set_xlabel('Time of day (hour)')
#     axs[i].set_xlim(start_time, end_time)
#     axs[i].set_ylim(bottom=0)
#     
# plt.savefig(f'../img/{image_folder}/uxsim_data_{image_name}_{plot_run}.png', dpi=300, bbox_inches='tight')

In [ ]:
# Merge into one df
long_uxsim_df = pd.concat(uxsim_dict, axis=0, keys=uxsim_dict.keys(), names=["Run"]).stack().reset_index()
long_uxsim_df.columns = ['Run', 'time_bin', 'area', 'variable', 'value']

# Plot
fig, axs = plt.subplots(3, 2, figsize=(14, 14))
axs = axs.flatten()
errorbar = ("pi", 50)

for i, variable in enumerate(uxsim_dict[runs[0]].columns):
    sns.lineplot(data=long_uxsim_df[long_uxsim_df['variable'] == variable], x='time_bin', y='value',  ax=axs[i], errorbar=errorbar, hue='Run')
    axs[i].set_title(f"{variable} in different areas")
    axs[i].set_ylabel(variable)
    axs[i].set_xlabel('Time of day (hour)')
    axs[i].set_xlim(start_time, end_time)
    axs[i].set_ylim(bottom=0)
# Add a legend
labels = [[f'Run {run}', f"{errorbar[1]}% {errorbar[0]}"] for run in runs]
flat_labels = [item for sublist in labels for item in sublist]
fig.legend(title='Run', loc=(0.9, 0.5), labels=flat_labels)

plt.savefig(f'../img/{image_folder}/uxsim_data_{image_name}.png', dpi=300, bbox_inches='tight')

In [ ]:
long_uxsim_df_city = long_uxsim_df[long_uxsim_df['area'].isin(city_areas)]

min_vars = ['average_speed']
max_vars = ['average_delay', 'total_travel_time', 'traffic_volume', 'vehicle_density', 'vehicles_remain']

# For variables in min_vars, apply the 'min' function
min_df = long_uxsim_df_city[long_uxsim_df_city['variable'].isin(min_vars)].copy()

# For variables in max_vars, apply the 'max' function
max_df = long_uxsim_df_city[long_uxsim_df_city['variable'].isin(max_vars)].copy()

min_aggregated = min_df.groupby(['Run', 'area', 'variable'], as_index=False)['value'].min()
min_aggregated.rename(columns={'value': 'aggregated_value'}, inplace=True)

max_aggregated = max_df.groupby(['Run', 'area', 'variable'], as_index=False)['value'].max()
max_aggregated.rename(columns={'value': 'aggregated_value'}, inplace=True)

aggregated_df = pd.concat([min_aggregated, max_aggregated], ignore_index=True)
aggregated_df['aggregated_value'] = aggregated_df['aggregated_value'].fillna(0)

In [ ]:
# Create heatmaps
fig, axs = plt.subplots(6, 1, figsize=(10, 12))
axs = axs.flatten()
vars = min_vars + max_vars
plt.tight_layout()

for i, variable in enumerate(vars):
    agg_method = 'min' if variable in min_vars else 'max'
    # Filter the DataFrame for the current variable
    df_filtered = aggregated_df[aggregated_df['variable'] == variable]

    # Pivot the DataFrame to get 'Run' as rows and 'area' as columns
    heatmap_data = df_filtered.pivot(index='Run', columns='area', values='aggregated_value')

    # Plot the heatmap using seaborn. Red-green color map is used. If max, reverse the color map.
    color_map = 'RdYlGn_r' if agg_method == 'max' else 'RdYlGn'
    sns.heatmap(heatmap_data, ax=axs[i], cmap=color_map, cbar_kws={'label': variable}, vmin=0, square=True)

    # Set title for each heatmap
    axs[i].set_title(f'{variable} ({agg_method})')
    
    # For the last subplot, use full area names with numbers as x-tick labels
    if i == len(vars) - 1:
        full_area_labels = [f'{area_names[area]} ({area})' for area in city_areas]
        axs[i].set_xticklabels(full_area_labels, rotation=60, ha='right')

# Save the figure
plt.suptitle('Heatmaps for min and max values of different variables in different areas')
plt.tight_layout()
plt.savefig(f'../img/{image_folder}/uxsim_heatmaps_{image_name}.png', dpi=300, bbox_inches='tight')

### Parking data visualization

In [ ]:
# Convert parked_dict to DataFrame
parked_df = pd.DataFrame(parked_dict)
# Long form
long_parked_df = parked_df.stack().reset_index()
# Rename columns
long_parked_df.columns = ['area', 'time', 'value']
long_parked_df = long_parked_df.set_index(['time', 'area'])

# Normalize the values on the first time step
long_parked_df['parked_norm'] = long_parked_df.groupby('area')['value'].transform(lambda x: x / x.iloc[0])
long_parked_df.head(3)

In [ ]:
gdf_mrdh_65 = pd.read_pickle("../data/areas_mrdh_weighted_centroids.pkl")

In [ ]:
with open("../data/polygons.pkl", "rb") as f:
    city_polygon, area_polygon = pickle.load(f)

city_polygon_series = gpd.GeoSeries(city_polygon, crs="epsg:4326")
city_polygon_series = city_polygon_series.to_crs(epsg=28992)
gdf_mrdh_65["in_city"] = gdf_mrdh_65.centroid.within(city_polygon_series.geometry[0])
gdf_mrdh_65 = gdf_mrdh_65[gdf_mrdh_65["in_city"]]

In [ ]:
# Create a dictionary mapping the index to the area of gdf_mrdh_65
area_dict = (gdf_mrdh_65.area / 1000000).to_dict()

# Calculate the parked cars per area
long_parked_df["parked_density"] = long_parked_df["value"] / long_parked_df.index.get_level_values("area").map(area_dict)
long_parked_df.head(3)

In [ ]:
# Plot
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
axs = axs.flatten()
for i, col in enumerate(['value', 'parked_norm', 'parked_density']):
    sns.lineplot(data=long_parked_df, x='time', y=col, hue='area', palette='rocket', ax=axs[i])
    axs[i].set_title(f'Parked cars in different areas ({col})')
    axs[i].set_ylabel('Parked cars')
    axs[i].set_xlabel('Time of day (hour)')
    axs[i].set_xlim(start_time, end_time)
    if col == 'value':
        axs[i].set_ylim(bottom=0)

# Save image
plt.savefig(f'../img/{image_folder}/parked_data_{image_name}.png', dpi=300, bbox_inches='tight')